
## 메가박스에서 영화에 대한 코드를 따오기 위해서 크롤링 진행
테스트 삼아 25000000 ~ 25005000와 24070000 ~ 24075000 


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

headers = {"User-Agent": "Mozilla/5.0"}
base = "25" # 자신의 범위 앞자리 두 코드의 범위에 따라 조정해주세요.

# 범위 설정은 뒷자리를 보고 진행해야 하는데요 예시 보고 참고해서 범위 지정해주세요
# 24070000 ~ 24075000 해당 범위일 경우
# base = "24"로 설정하고
# code_range = range(70000, 75000 + 1, 100) # 24070000 ~ 24075000
code_range = range(0, 5000 + 1, 100)  # 25000000 ~ 25005000
movie_codes = [f"{base}{str(i).zfill(6)}" for i in code_range]

movies_found = []

for code in movie_codes:
    url = f"https://www.megabox.co.kr/movie-detail?rpstMovieNo={code}"
    try:
        res = requests.get(url, headers=headers, timeout=5)
        if res.status_code == 200:
            soup = BeautifulSoup(res.text, "html.parser")
            title_tag = soup.select_one("div.movie-detail-cont > p.title")
            if title_tag:
                title = title_tag.text.strip()
                movies_found.append({"영화코드": code, "영화제목": title})
                print(f"✅ {code} → {title}")
            else:
                print(f"⚠️ {code} → 제목 없음")
        else:
            print(f"❌ {code} → 응답 코드 {res.status_code}")
    except Exception as e:
        print(f"⛔ {code} → 오류 발생: {str(e)}")
    time.sleep(0.7)

df = pd.DataFrame(movies_found)
df.to_csv("megabox_movie_list.csv", index=False, encoding="utf-8-sig")
print("\n🎉 저장 완료! → megabox_movie_list.csv")


## 메가박스 영화 코드에 맞게 세부 정보(장르, 상영시간, 등급, 개봉일) 저장 및 NAN 열이 포함된 행은 삭제하는 전처리

In [ ]:
import pandas as pd
import time
import random
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# Selenium 브라우저 옵션 설정 (headless는 ❌)
options = Options()
options.add_argument("user-agent=Mozilla/5.0")

# 크롬 드라이버 실행
driver = webdriver.Chrome(options=options)

# 영화 목록 CSV 불러오기
df = pd.read_csv("megabox_movie_list.csv") # 혹시 파일 이름이 안 맞으면 전 단계예서 수행했던 파일 이름 사용하면 됩니다
#앞서서 크롤링한 파일이 두 개니까 여기도 이름 바꿔서 한 번 더 진행
df["장르"] = ""
df["상영시간"] = ""
df["등급"] = ""
df["개봉일"] = ""

for idx, row in df.iterrows():
    code = str(row["영화코드"])
    title = row["영화제목"]
    url = f"https://www.megabox.co.kr/movie-detail?rpstMovieNo={code}"

    try:
        driver.get(url)

        # 영화 정보 로딩 대기
        WebDriverWait(driver, 7).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div.movie-info.infoContent"))
        )

        # HTML 파싱
        soup = BeautifulSoup(driver.page_source, "html.parser")
        info_box = soup.select_one("div.movie-info.infoContent")

        if info_box:
            p_tags = info_box.find_all("p")
            genre = running_time = rating = release_date = ""

            for p in p_tags:
                text = p.get_text(strip=True).replace("\xa0", "")
                if ":" not in text:
                    continue

                label, value = [x.strip() for x in text.split(":", 1)]

                if label == "장르":
                    if "/" in value:
                        genre, running_time = [x.strip() for x in value.split("/", 1)]
                    else:
                        genre = value
                elif label == "등급":
                    rating = value
                elif label == "개봉일":
                    release_date = value

            # 결과 저장
            df.at[idx, "장르"] = genre
            df.at[idx, "상영시간"] = running_time
            df.at[idx, "등급"] = rating
            df.at[idx, "개봉일"] = release_date

            # 로그 출력
            print(f"\n✅ [{title}]")
            print(f"장르: {genre}")
            print(f"상영시간: {running_time}")
            print(f"등급: {rating}")
            print(f"개봉일: {release_date}")
        else:
            print(f"⚠️ [{title}] → 정보 div를 찾지 못함")

    except Exception as e:
        print(f"❌ [{title}] 에러 발생: {e}")

    time.sleep(random.uniform(0.8, 1.5))  # 서버 부하 방지

# 크롤링 결과 저장
df.to_csv("megabox_movie_list_detail.csv", index=False, encoding="utf-8-sig") # 앞서서 불러오는 게 달라졌으면 여기도 이름 바꿔주셔야 해용
driver.quit()
print("\n🎉 크롤링 완료! megabox_movie_list_detail.csv에 저장됨") # 안 헷갈리게 여기도요!


In [ ]:
import pandas as pd

# 1. 파일 불러오기
df1 = pd.read_csv("megabox_movie_list_detail.csv")
df2 = pd.read_csv("megabox_movie_list_detail2.csv")
# 2. 공백 문자열("")을 NaN으로 처리 후, 주요 컬럼 기준 결측 행 제거
df1_cleaned = df1.replace("", pd.NA).dropna(subset=["장르", "상영시간", "등급", "개봉일"])
df2_cleaned = df2.replace("", pd.NA).dropna(subset=["장르", "상영시간", "등급", "개봉일"])

# 3. 원래 파일명에 덮어쓰기 저장
df1_cleaned.to_csv("megabox_movie_list_detail.csv", index=False, encoding="utf-8-sig")
df2_cleaned.to_csv("megabox_movie_list_detail2.csv", index=False, encoding="utf-8-sig")
print(f"\n🧹 전처리 완료: {len(df1) - len(df1_cleaned)}개 행 제거됨")
print("✅ 저장 파일: megabox_movie_list_detail.csv (덮어쓰기)")
print(f"\n🧹 전처리 완료: {len(df2) - len(df2_cleaned)}개 행 제거됨")
print("✅ 저장 파일: megabox_movie_list_detail2.csv (덮어쓰기)")

## 리뷰 5개 크롤링, 만약 리뷰가 5개 이하일 경우 대상 영화의 행을 지우는 작업까지 진행

In [ ]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

# 크롬 옵션 설정
options = Options()
options.add_argument("log-level=3")
driver = webdriver.Chrome(options=options)

# CSV 불러오기
df = pd.read_csv("megabox_movie_list_detail.csv") #여기에서 파일 이름 바꾸면 됩니다

# 리뷰 컬럼 5개 미리 추가
for i in range(1, 6):
    df[f"리뷰{i}"] = ""

# 결과를 저장할 새로운 리스트
filtered_rows = []

for idx, row in df.iterrows():
    movie_code = str(row["영화코드"])
    title = row["영화제목"]
    url = f"https://www.megabox.co.kr/movie-detail/comment?rpstMovieNo={movie_code}"

    try:
        driver.get(url)
        time.sleep(2)

        soup = BeautifulSoup(driver.page_source, "html.parser")
        review_tags = soup.select("div.story-txt")

        # 리뷰가 5개 이상인 경우만 저장
        if len(review_tags) >= 5:
            for i in range(5):
                row[f"리뷰{i+1}"] = review_tags[i].text.strip()
            filtered_rows.append(row)
            print(f"✅ [{title}] 리뷰 5개 수집 완료")
        else:
            print(f"⚠️ [{title}] 리뷰 부족 → 삭제 대상")

    except Exception as e:
        print(f"❌ [{title}] 에러 발생: {e}")

    time.sleep(1.0)

driver.quit()

# 필터링된 데이터프레임 생성
df_filtered = pd.DataFrame(filtered_rows)

# 저장
df_filtered.to_csv("megabox_movie_list_with_reviews.csv", index=False, encoding="utf-8-sig") #저장되는 파일명도 1, 2 맞춰서 바꾸세용
print(f"\n🎉 리뷰 5개 이상 영화만 저장 완료! 최종 {len(df_filtered)}편")
